---
# 12. Union Find
|Problem|Dfficulty|Link|
|--------|--------|-----------|
|128. Longest Consecutive Sequence | <span style="color:yellow">Medium</span> | https://leetcode.com/problems/longest-consecutive-sequence/description |
|684. Redundant Connection|  <span style="color:yellow">Medium</span>  | https://leetcode.com/problems/redundant-connection/description |
|721. Accounts Merge |  <span style="color:yellow">Medium</span>   | https://leetcode.com/problems/accounts-merge/description |
|839. Similar String Groups | <span style="color:red">Hard</span> | https://leetcode.com/problems/similar-string-groups/description |
|1254. Number of Closed Islands | <span style="color:yellow">Medium</span>  | https://leetcode.com/problems/number-of-closed-islands/description |


# Union Find Class 

```cpp  
class UnionFind {
public:
    UnionFind(int n) : parent(n), rank(n, 0) {
        for (int i = 0; i < n; ++i) parent[i] = i;    
    }

    int find(int x) {
        if (parent[x] != x) parent[x] = find(parent[x]);
        
        return parent[x];
    }

    void unionSets(int x, int y) {
        int rootX = find(x);
        int rootY = find(y);

        if (rootX != rootY) {
            if (rank[rootX] > rank[rootY]) parent[rootY] = rootX;

            else if (rank[rootX] < rank[rootY]) parent[rootX] = rootY;

            else {
                parent[rootY] = rootX;
                rank[rootX]++;
            }
        }
    }

private:
    vector<int> parent;
    vector<int> rank;
};
```

---
# 128. Longest Consecutive Sequence

# Intuition
Use a Union-Find data structure to group consecutive numbers together. By merging sets of consecutive numbers, we can determine the size of the largest set, which corresponds to the longest consecutive sequence.

# Approach
1. **Union-Find Data Structure**:
   - Initialize a Union-Find (Disjoint Set Union, DSU) class to manage the merging and finding of sets.
   - Each number in the array will be treated as a node in the Union-Find structure.

2. **Mapping Numbers to Indices**:
   - Use an unordered map to keep track of the index of each unique number in the array.
   - This helps in efficiently finding and merging consecutive numbers.

3. **Union Operation for Consecutive Numbers**:
   - For each number, if it hasn't been processed yet, add it to the map.
   - Check if the number - 1 and number + 1 exist in the map. If they do, merge the sets containing these numbers with the set containing the current number.

4. **Find the Longest Consecutive Sequence**:
   - After processing all numbers, use another unordered map to count the size of each set (rooted at the same parent).
   - The maximum size among these sets is the length of the longest consecutive sequence.

# Complexity

## Time complexity
- `O(n log n)` in worst case
- `O(n)` in average

## Space complexity
- `O(n)`

```cpp
class Solution {
public:
    int longestConsecutive(vector<int>& nums) {
        if (nums.empty()) return 0;

        int n = nums.size();
        UnionFind uf(n);
        unordered_map<int, int> numToIndex;

        for (int i = 0; i < n; ++i) {
            if (numToIndex.find(nums[i]) == numToIndex.end()) {
                numToIndex[nums[i]] = i;
                if (numToIndex.find(nums[i] - 1) != numToIndex.end()) uf.unionSets(i, numToIndex[nums[i] - 1]);
                
                if (numToIndex.find(nums[i] + 1) != numToIndex.end()) uf.unionSets(i, numToIndex[nums[i] + 1]);
            }
        }

        unordered_map<int, int> rootToCount;
        int maxLength = 0;

        for (int i = 0; i < n; ++i) {
            int root = uf.find(i);
            rootToCount[root]++;
            maxLength = max(maxLength, rootToCount[root]);
        }

        return maxLength;
    }
};
```

---
# 684. Redundant Connection

# Intuition
Use the Union-Find (Disjoint Set Union, DSU) data structure. The redundant connection is the one that, if added, creates a cycle in the graph.

# Approach
1. **Union-Find Data Structure**:
   - Initialize the Union-Find class to manage the merging and finding of sets. Each node will be treated as a node in the Union-Find structure.

2. **Initialization**:
   - Initialize the `parent` and `rank` arrays in the Union-Find class to represent each node as its own parent initially, and set the rank of each node to 0.

3. **Union Operation with Cycle Detection**:
   - For each edge, use the union operation to try and merge the sets containing the two nodes of the edge.
   - If the union operation detects that both nodes are already in the same set (i.e., they have the same root), then adding this edge would create a cycle. This edge is the redundant connection.

4. **Return the Redundant Edge**:
   - The first edge that, when processed, indicates a cycle (i.e., nodes already in the same set) is returned as the redundant connection.

# Complexity

## Time complexity
- `O(n)`, where `n` is the number of edges. Each union and find operation is nearly constant time due to path compression and union by rank.

## Space complexity
- `O(n)`, for storing the parent and rank arrays.

```cpp
class Solution {
public:
    vector<int> findRedundantConnection(vector<vector<int>>& edges) {
        int n = edges.size();
        UnionFind uf(n);

        for (const auto& edge : edges) {
            if (!uf.unionSets(edge[0], edge[1])) {
                return edge;  // 순환이 발생한 간선 반환
            }
        }

        return {};
    }
};

---
# 721. Accounts Merge

# Intuition
Each account can be represented as a set of emails belonging to a user. If two accounts share at least one email, they should be merged.

# Approach
1. **Union-Find Data Structure**:
   - Use the Union-Find class to manage the merging and finding of sets of accounts based on shared emails.

2. **Mapping Emails to Accounts**:
   - Use two hash maps:
     - `emailToIndex` to map each email to the index of the account it belongs to.
     - `emailToName` to map each email to the corresponding user's name.

3. **Union Operation for Emails**:
   - For each account, iterate through its emails.
   - If an email is seen for the first time, map it to the current account's index.
   - If an email has been seen before, union the current account's index with the previously mapped index of that email.

4. **Group Emails by Account**:
   - Use another hash map `indexToEmails` to group emails by their root account index (after union operations).

5. **Generate Merged Accounts**:
   - For each group of emails, sort the emails and prepend the user's name.
   - Collect all such groups into the result list `mergedAccounts`.

# Complexity

## Time complexity
- `O(nk log nk)`, where `n` is the number of accounts and `k` is the maximum number of emails per account. 

## Space complexity
- `O(nk)`

```cpp
class Solution {
public:
    vector<vector<string>> accountsMerge(vector<vector<string>>& accounts) {
        int n = accounts.size();
        UnionFind uf(n);
        unordered_map<string, int> emailToIndex;
        unordered_map<string, string> emailToName;

        for (int i = 0; i < n; ++i) {
            string name = accounts[i][0];
            for (int j = 1; j < accounts[i].size(); ++j) {
                string email = accounts[i][j];
                if (emailToIndex.find(email) == emailToIndex.end()) {
                    emailToIndex[email] = i;
                    emailToName[email] = name;
                } 
                else uf.unionSets(i, emailToIndex[email]);
                
            }
        }
        
        unordered_map<int, vector<string>> indexToEmails;
        
        for (const auto& email_index_pair : emailToIndex) {
            string email = email_index_pair.first;
            int index = uf.find(email_index_pair.second);
            indexToEmails[index].push_back(email);
        }

        // generate result
        vector<vector<string>> mergedAccounts;
        for (const auto& index_emails_pair : indexToEmails) {
            int index = index_emails_pair.first;
            vector<string> emails = index_emails_pair.second;
            sort(emails.begin(), emails.end());
            emails.insert(emails.begin(), emailToName[emails[0]]);
            mergedAccounts.push_back(emails);
        }

        return mergedAccounts;
    }
};


---
# 839. Similar String Groups

# Intuition
Two strings are similar if they can be made the same by swapping at most two characters. 

# Approach
1. **Union-Find Data Structure**:
   - Use Union-Find to manage and merge groups of similar strings.
   - Initialize parent (`p`) and rank (`r`) arrays to manage the sets.

2. **Initialize Union-Find**:
   - Each string starts as its own set. Initialize the parent array where each node is its own parent and rank array to manage the tree heights.
   - Initialize `sum` to keep track of the number of distinct groups.

3. **Check Similarity**:
   - Implement a function `similar` to check if two strings are similar (i.e., they differ by at most two characters).

4. **Union Operation**:
   - For each pair of strings, if they are similar, union their sets.
   - The union operation involves finding the roots of the sets of both strings and merging them if they are different, decrementing the number of groups (`sum`) if a merge happens.

5. **Return the Number of Groups**:
   - After processing all pairs, the `sum` variable contains the number of distinct groups.

# Complexity

## Time complexity
- `O(n^2 * k)`, where `n` is the number of strings and `k` is the maximum length of a string. 
## Space complexity
- `O(n)`

```cpp
class Solution {
public:
    int p[300], r[300], sum;
    int numSimilarGroups(vector<string>& strs) {
        int n = strs.size();
        sum = n;
        for (int i = 0; i < n; i++) p[i] = i, r[i] = 1;
        for (int i = 0; i < n; i++) {
            for (int j = i + 1; j < n; j++) {
                if (similar(strs[i], strs[j])) {
                    Union(i, j);
                }
            }
        }
        return sum;
    }
    
    bool similar(string const& s1, string const& s2) {
        int diff = 0;
        for (int i = 0; i < s1.size(); i++) {
            if(s1[i] != s2[i]) {
                diff++;
                if (diff > 2) return false;
            }
        }
        return diff <= 2;
    }
    
    void Union(int i, int j) {
        int p1 = find(i);
        int p2 = find(j);
        if (p1 == p2) return;
        if (r[p1] < r[p2]) swap(p1, p2);
        p[p2] = p1;
        r[p1] += r[p2];
        sum--;
    }
    
    int find(int i) {
        while (i != p[i]) {
            p[i] = p[p[i]];
            i = p[i];
        }
        return i;
    }
};

---
# 1254. Number of Closed Islands

# Intuition
A closed island is a group of connected zeros (0s) that is completely surrounded by ones (1s) and does not touch the boundary of the grid. 
Use Union-Find data structure to manage the connected components

# Approach
1. **Union-Find Data Structure**:
   - Use Union-Find to manage the merging of groups of connected zeros.
   - Initialize `parent` and `rank` arrays to manage the sets.

2. **Initialization**:
   - Initialize the `parent` array such that each cell is its own parent.
   - Initialize the `rank` array to 0 for all cells.

3. **Union Operation for Zeros**:
   - For each cell in the grid, if it contains a zero, union it with its neighboring zeros (up, down, left, right).
   - This creates groups of connected zeros.

4. **Track Boundary Islands**:
   - Traverse the grid and add the root of each zero that touches the boundary to a set `boundaryIslands`.

5. **Count Closed Islands**:
   - Traverse the grid again and count the number of unique groups of zeros whose roots are not in `boundaryIslands`.

# Complexity

## Time complexity
- `O(m * n)`, where `m` is the number of rows and `n` is the number of columns in the grid. Each cell is processed a constant number of times.

## Space complexity
- `O(m * n)`, for storing the parent and rank arrays, and the sets used to track islands.

```cpp
class Solution {
public:
    vector<int> parent;
    vector<int> rank;

    int find(int x) {
        if (parent[x] != x) {
            parent[x] = find(parent[x]);
        }
        return parent[x];
    }

    void unionSets(int x, int y) {
        int rootX = find(x);
        int rootY = find(y);

        if (rootX != rootY) {
            if (rank[rootX] > rank[rootY]) {
                parent[rootY] = rootX;
            } else if (rank[rootX] < rank[rootY]) {
                parent[rootX] = rootY;
            } else {
                parent[rootY] = rootX;
                rank[rootX]++;
            }
        }
    }

    int closedIsland(vector<vector<int>>& grid) {
        int rows = grid.size();
        int cols = grid[0].size();
        parent.resize(rows * cols);
        rank.resize(rows * cols, 0);

        for (int i = 0; i < rows * cols; ++i) parent[i] = i;

        for (int r = 0; r < rows; ++r) {
            for (int c = 0; c < cols; ++c) {
                if (grid[r][c] == 0) {
                    int index = r * cols + c;
                    if (r > 0 && grid[r - 1][c] == 0) unionSets(index, (r - 1) * cols + c);
                    
                    if (c > 0 && grid[r][c - 1] == 0) unionSets(index, r * cols + (c - 1));
                    
                    if (r < rows - 1 && grid[r + 1][c] == 0) unionSets(index, (r + 1) * cols + c);
                    
                    if (c < cols - 1 && grid[r][c + 1] == 0) unionSets(index, r * cols + (c + 1));
                }
            }
        }

        // eliminate islands process
        unordered_set<int> boundaryIslands;
        for (int r = 0; r < rows; ++r) {
            for (int c = 0; c < cols; ++c) {
                if (grid[r][c] == 0) {
                    int index = r * cols + c;
                    if (r == 0 || c == 0 || r == rows - 1 || c == cols - 1) boundaryIslands.insert(find(index));
                }
            }
        }

        // counting remain groups
        unordered_set<int> allIslands;
        for (int r = 0; r < rows; ++r) {
            for (int c = 0; c < cols; ++c) {
                if (grid[r][c] == 0) {
                    int index = r * cols + c;
                    int root = find(index);
                    if (boundaryIslands.find(root) == boundaryIslands.end()) {
                        allIslands.insert(root);
                    }
                }
            }
        }

        return allIslands.size();
    }
};